산사태 발생 지역과 ASOS관측소를 거리 기준으로 매칭하여 데이터 셋을 구축한다.  
그전에 기상 데이터 전처리를 진행한다  
기상 데이터의 columns을 살펴보고 학습에 사용할 변수를 선택하는 작업이다.  
사건이 발생한 시각을 기록한 컬럼은 산사태 발생과 큰 연관이 없을 것이라 생각하여 제외한다.  
강수,적설 컬럼의 결측은 0으로 채운다  
결측이 50%이상인 컬럼은 제외한다.  

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import folium
import warnings
import datetime
warnings.filterwarnings(action='ignore')

### 데이터 로드

In [38]:
# ASOS 관측 데이터
weather = pd.read_csv('../data/기상/OBS_ASOS_DD_20210601231517.csv', encoding = 'cp949')
# ASOS 관측소 메타 데이터
coor = pd.read_csv('../data/기상/META_관측지점정보_20210603224446.csv',encoding='cp949')
# 산사태 발생 이력 데이터
land = pd.read_csv('../data/산사태 발생이력.csv', encoding = 'cp949')
# 행정동 경계 데이터
boundary = gpd.read_file('../data/행정동 경계/umd.shp',encoding='utf-8')

### 기상 데이터 전처리

In [39]:
# 기상 데이터 전처리
# 시각 컬럼 제거
clock_col = [i for i in weather.columns if '시각' in i]
weather = weather.drop(clock_col,axis=1)
# 강수 컬럼 결측 0으로 채우기
rainfall_col = [i for i in weather.columns if '강수' in i]
weather[rainfall_col] = weather[rainfall_col].fillna(0)
# 적설 컬럼 결측 0으로 채우기
snow_col = [i for i in weather.columns if '적설' in i]
weather[snow_col] = weather[snow_col].fillna(0)
# 결측 많은 컬럼 제거
many_nan = [i for i in weather.columns if weather.isna().sum()[i]/len(weather)>0.5]
weather = weather.drop(many_nan, axis=1)

### 기상 데이터와 산사태 발생 데이터 결합

In [40]:
# 기상청 메타 데이터를 geopandas가 인식할 수 있도록 point 형태로 변환
coor['geometry'] = gpd.points_from_xy(coor.경도,coor.위도)
tmp = gpd.GeoDataFrame({'geometry':coor['geometry']})

# 기상청 페이지에 기재된대로 좌표계 형식 설정 
tmp.crs = {'init':'epsg:4326'}

# 행정동 경계 데이터도 같은 좌표계로 설정해준다.
boundary = boundary.to_crs({'init':'epsg:4326'})

In [41]:
# 산사태가 발생한 지역의 polygon데이터만 수집
land_bound = pd.DataFrame()
for i in set(land['umd']):
    land_bound = pd.concat([land_bound,boundary[boundary['ADM_DR_NM']==i.strip(' ')]],axis=0)
land_bound = land_bound.reset_index().drop('index',axis=1)

In [42]:
# 산사태가 발생한 지역에 속한 ASOS 관측소 찾기
coor['land'] = 0
land_bound['weather'] = 0
for i in range(len(coor)):
    for j in range(len(land_bound)):
        if land_bound['geometry'].iloc[j].contains(coor['geometry'].iloc[i]):
            print(coor['지점명'].iloc[i], land_bound['ADM_DR_NM'].iloc[j])
            coor['land'].iloc[i] = 1
            land_bound['weather'].iloc[j] = 1

울릉도 울릉읍
울진 울진읍
울진 울진읍
통영 정량동
진주 판문동
진주 초장동
북창원 중앙동
창원시 중앙동
의령군 의령읍
함양군 함양읍
영덕 영해면
경주시 선도동
거창 거창읍
거창 거창읍
합천 합천읍
산청 산청읍
남해 이동면


ASOS 관측소가 행정구역에 모두 속할 줄 알았는데 그렇지 않았다.  
거리를 계산해서 가까운 지역과 관측소 매칭

교체된 관측기에 대한 정보를 정제하고 다시 매칭

In [43]:
# 종료일이 존재하는 관측소를 검색
coor['종료일'] = coor['종료일'].astype('str')
coor[coor['종료일']!='nan']

,지점,시작일,종료일,지점명,지점주소,관리관서,위도,경도,노장해발고도(m),기압계(관측장비지상높이(m)),기온계(관측장비지상높이(m)),풍속계(관측장비지상높이(m)),강우계(관측장비지상높이(m)),geometry,land
5,99,2001-12-07,2013-10-22,문산,경기도 파주시문산읍 마정로46-29(파주기상대),파주기상대(99),37.8859,126.7665,29.42,31.40,1.70,10.0,0.5,POINT (126.76650 37.88590),0
7,100,1971-07-15,2006-11-06,대관령,강원도 평창군대관령면 경강로5372 대관령자동기상관측소,NaN,37.6869,128.7587,842.52,843.96,1.50,10.0,1.6,POINT (128.75870 37.68690),0
10,102,2000-11-01,2018-05-01,백령도,인천광역시 옹진군백령면 진촌리1031,백령도기상대(102),37.9661,124.6305,146.00,147.20,1.80,9.0,1.2,POINT (124.63050 37.96610),0
15,108,1907-10-01,2010-08-15,서울,서울특별시 종로구송월길 52서울기상관측소,서울기상관측소(108),37.5714,126.9658,85.50,86.50,1.50,10.0,0.6,POINT (126.96580 37.57140),0
17,112,1904-08-29,2013-10-22,인천,인천광역시 중구전동 25번지인천기상대,인천기상대(112),37.4776,126.6244,68.15,70.15,1.50,10.0,0.6,POINT (126.62440 37.47760),0
20,116,1984-01-01,1990-12-31,관악산,경기도 과천시중앙로 관악산길(관악산기상레이더관측소),NaN,37.4442,126.9639,626.76,627.96,1.40,3.9,1.1,POINT (126.96390 37.44420),0
22,119,1964-01-01,2019-07-24,수원,경기도 수원시권선구 권선로276,수도권기상청(119),37.2723,126.9853,34.84,35.84,1.60,18.7,1.1,POINT (126.98530 37.27230),0
27,130,1971-01-12,2020-12-04,울진,경상북도 울진군울진읍 현내항길157 울진지역기상서비스센터,대구(구 143),36.9918,129.4128,48.98,50.18,1.75,10.0,1.3,POINT (129.41280 36.99180),1
32,136,1973-01-01,1978-05-01,안동,경상북도 안동시열루재1길 16(운안동433-1) 안동기상대,NaN,36.5730,128.7073,139.39,141.40,1.50,10.0,0.6,POINT (128.70730 36.57300),0
36,140,1968-01-01,2003-12-01,군산,전라북도 군산시내흥동 425-10호군산지역기상서비스센터,NaN,35.9930,126.7057,25.57,30.74,1.50,18.0,0.6,POINT (126.70570 35.99300),0


산사태 발생 이력 데이터의 시작이 2011년 7월 9일  
이전에 관측을 중단한 관측소는 데이터에서 제외한다  

In [44]:
coor = coor.drop([4,8,9,12,15,32],axis=0)

In [45]:
# 데이터 정제 후 다시 매칭
# 모든 관측소와 거리를 계산하여 가장 가까운 관측소를 매칭
land_bound['ASOS'] = 0
for i in range(len(land_bound)):
    dis = []
    for j in range(len(coor)):
        dis.append(land_bound['geometry'].iloc[i].distance(coor['geometry'].iloc[j]))
    land_bound['ASOS'].iloc[i] = coor['지점명'].iloc[dis.index(min(dis))]

In [46]:
# 하나의 관측소가 커버하는 지역의 개수
for i in set(land_bound['ASOS']):
    print(i,len(land_bound[land_bound['ASOS']==i]))

합천 23
밀양 18
대구 3
경주시 6
창원시 1
울산 2
울진 8
북창원 7
영주 1
김해시 7
울릉도 3
거창 14
문경 1
함양군 10
봉화 1
진주 24
창원 2
영천 5
구미 12
남해 9
통영 12
영덕 12
산청 11
장수 1
추풍령 11
양산시 7
안동 2
광양시 6
포항 19
상주 4
의령군 4
마산 10
청송군 4


In [47]:
# 산사태 발생 지역에 데이터 프레임 하나씩

In [48]:
land_dict = {k.strip(' '):[] for k in land['umd']}
# 다음 지역에 대해선 예외처리한 후 직접 데이터프레임 병합
except_land = []
for i in land_dict.keys():
    if len(land_bound[land_bound['ADM_DR_NM']==i]) > 1:
        print(i,list(land_bound[land_bound['ADM_DR_NM']==i]['ASOS']))
        except_land.append(i)

상동면 ['김해시', '밀양']
성산면 ['합천', '밀양']
묘산면 ['합천', '합천']
병곡면 ['영덕', '함양군']
대가면 ['구미', '진주']
남면 ['구미', '남해', '구미', '남해']
서면 ['영천', '울릉도', '남해']
북면 ['울진', '울릉도', '북창원']
봉산면 ['추풍령', '합천']
덕곡면 ['합천', '합천']
동해면 ['포항', '마산']
자산동 ['추풍령', '마산']
북부동 ['대구', '김해시']
도산면 ['안동', '통영']
중앙동 ['포항', '영천', '대구', '진주', '통영', '양산시', '북창원']


In [49]:
land['date'] = land['date'].apply(lambda X:datetime.datetime(int(str(X)[:4]),int(str(X)[4:6]),int(str(X)[6:])))
weather['일시'] = pd.to_datetime(weather['일시'])

In [50]:
for i in land_dict.keys():
    if i in except_land:
        pass
    else:
        asos = land_bound[land_bound['ADM_DR_NM']==i]['ASOS'].iloc[0]
        asosDF = weather[weather['지점명']==asos].sort_values('일시')
        landDF = land[land['umd']==i]
        land_dict[i] = pd.merge(landDF,asosDF, left_on='date', right_on='일시', how='outer')
        land_dict[i]['sd'] = land_dict[i]['sd'].iloc[0]
        land_dict[i]['sgg'] = land_dict[i]['sgg'].iloc[0]
        land_dict[i]['umd'] = land_dict[i]['umd'].iloc[0]
        land_dict[i]['sum_cnt'] = land_dict[i]['sum_cnt'].fillna(0)
        land_dict[i]['sum_hpa'] = land_dict[i]['sum_hpa'].fillna(0)        

In [51]:
land_dict['내일동'].corr()

,sum_cnt,sum_hpa,지점,평균기온(°C),최저기온(°C),최고기온(°C),강수 계속시간(hr),10분 최다 강수량(mm),1시간 최다강수량(mm),일강수량(mm),...,최저 해면기압(hPa),평균 해면기압(hPa),가조시간(hr),합계 일조시간(hr),일 최심신적설(cm),일 최심적설(cm),합계 3시간 신적설(cm),평균 지면온도(°C),최저 초상온도(°C),9-9강수(mm)
sum_cnt,1.000000,1.000000,NaN,0.016350,0.021497,0.011015,NaN,0.098954,0.167992,0.352671,...,-0.013585,-0.013216,0.023710,-0.028594,NaN,-0.000543,-0.000326,0.010557,0.023787,NaN
sum_hpa,1.000000,1.000000,NaN,0.016350,0.021497,0.011015,NaN,0.098954,0.167992,0.352671,...,-0.013585,-0.013216,0.023710,-0.028594,NaN,-0.000543,-0.000326,0.010557,0.023787,NaN
지점,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
평균기온(°C),0.016350,0.016350,NaN,1.000000,0.978159,0.970665,NaN,0.244041,0.224341,0.148540,...,-0.741911,-0.794934,0.839192,-0.067673,NaN,-0.055881,-0.026453,0.981743,0.963810,NaN
최저기온(°C),0.021497,0.021497,NaN,0.978159,1.000000,0.908852,NaN,0.288375,0.270524,0.206414,...,-0.750219,-0.803162,0.807042,-0.206417,NaN,-0.052347,-0.019963,0.945097,0.993013,NaN
최고기온(°C),0.011015,0.011015,NaN,0.970665,0.908852,1.000000,NaN,0.188187,0.164374,0.068978,...,-0.681000,-0.736140,0.820265,0.105355,NaN,-0.057774,-0.030312,0.973003,0.887942,NaN
강수 계속시간(hr),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10분 최다 강수량(mm),0.098954,0.098954,NaN,0.244041,0.288375,0.188187,NaN,1.000000,0.925168,0.702665,...,-0.271648,-0.270806,0.212166,-0.268618,NaN,-0.008113,-0.004867,0.197405,0.305878,NaN
1시간 최다강수량(mm),0.167992,0.167992,NaN,0.224341,0.270524,0.164374,NaN,0.925168,1.000000,0.820643,...,-0.274914,-0.265947,0.204969,-0.278212,NaN,-0.007877,-0.004725,0.177004,0.289244,NaN
일강수량(mm),0.352671,0.352671,NaN,0.148540,0.206414,0.068978,NaN,0.702665,0.820643,1.000000,...,-0.253228,-0.225606,0.136052,-0.342552,NaN,-0.008194,0.006761,0.092665,0.228771,NaN


예외처리한 지역에 대해 데이터셋 만들기

In [52]:
land[land['umd']=='상동면']

,date,sd,sgg,umd,sum_cnt,sum_hpa
5,2011-07-09,경상남도,밀양시,상동면,6,12.50
217,2016-10-05,경상남도,김해시,상동면,2,1.08


In [53]:
def make_DF(asos,ADM):
    asosDF = weather[weather['지점명']==asos].sort_values('일시')
    landDF = land[land['umd']==ADM]
    tmp = pd.merge(landDF,asosDF, left_on='date', right_on='일시', how='outer')
    tmp['sd'] = tmp['sd'].iloc[0]
    tmp['sgg'] = tmp['sgg'].iloc[0]
    tmp['umd'] = tmp['umd'].iloc[0]
    tmp['sum_cnt'] = tmp['sum_cnt'].fillna(0)
    tmp['sum_hpa'] = tmp['sum_hpa'].fillna(0)
    return tmp

In [54]:
land_dict['김해시 상동면'] = make_DF('김해시', '상동면')
land_dict['밀양 상동면'] = make_DF('밀양', '상동면')

land_dict['합천 성산면'] = make_DF('합천', '성산면')
land_dict['밀양 성산면'] = make_DF('밀양', '성산면')

land_dict['묘산면'] = make_DF('합천', '묘산면')
land_dict['덕곡면'] = make_DF('합천', '덕곡면')

land_dict['영덕 병곡면'] = make_DF('영덕', '병곡면')
land_dict['함양군 병곡면'] = make_DF('함양군', '병곡면')

land_dict['구미 대가면'] = make_DF('구미', '대가면')
land_dict['진주 대가면'] = make_DF('진주', '대가면')

land_dict['구미 남면'] = make_DF('구미', '남면')
land_dict['남해 남면'] = make_DF('남해', '남면')

land_dict['영천 서면'] = make_DF('영천', '서면')
land_dict['남해 서면'] = make_DF('남해', '서면')
land_dict['울릉도 서면'] = make_DF('울릉도', '서면')

land_dict['울진 북면'] = make_DF('울진', '북면')
land_dict['북창원 북면'] = make_DF('북창원', '북면')
land_dict['울릉도 북면'] = make_DF('울릉도', '북면')

land_dict['추풍령 봉산면'] = make_DF('추풍령', '봉산면')
land_dict['합천 봉산면'] = make_DF('합천', '봉산면')

land_dict['포항 동해면'] = make_DF('포항', '동해면')
land_dict['마산 동해면'] = make_DF('마산', '동해면')

land_dict['추풍령 자산동'] = make_DF('추풍령', '자산동')
land_dict['마산 자산동'] = make_DF('마산', '자산동')

land_dict['김해시 북부동'] = make_DF('김해시', '북부동')
land_dict['대구 북부동'] = make_DF('대구', '북부동')

land_dict['안동 도산면'] = make_DF('안동', '도산면')
land_dict['통영 도산면'] = make_DF('통영', '도산면')

land_dict['포항 중앙동'] = make_DF('포항', '중앙동')
land_dict['영천 중앙동'] = make_DF('영천', '중앙동')
land_dict['대구 중앙동'] = make_DF('대구', '중앙동')
land_dict['진주 중앙동'] = make_DF('진주', '중앙동')
land_dict['통영 중앙동'] = make_DF('통영', '중앙동')
land_dict['양산시 중앙동'] = make_DF('양산시', '중앙동')
land_dict['북창원 중앙동'] = make_DF('북창원', '중앙동')

In [55]:
remove_key = [i for i in land_dict.keys() if len(land_dict[i])==0]
for i in remove_key:
    del land_dict[i]